In [1]:
import re
from collections import defaultdict
import numpy as np
import tqdm
import math
import itertools

def count_ordered_selections0(idxseqs):
    if len(idxseqs) == 0:
        return 1
    if len(idxseqs) == 1:
        return len(idxseqs[0])
    if any(len(i) == 0 for i in idxseqs):
        return 0
    tot = 0
    for na in idxseqs[0]:
        idxseqs2 = [i[i>na] for i in idxseqs[1:]]
        tot += count_ordered_selections(idxseqs2)
    return tot



def above(i, na):
    while len(i) > 0 and i[0] <= na:
        i = i[1:]
    return i        

def count_ordered_selections1(idxseqs):
    if len(idxseqs) == 0:
        return 1
    if len(idxseqs) == 1:
        return len(idxseqs[0])
    if any(len(i) == 0 for i in idxseqs):
        return 0
    tot = 0
    for na in idxseqs[0]:
        idxseqs2 = [above(i, na) for i in idxseqs[1:]]
        tot += count_ordered_selections(idxseqs2)
    return tot


def count_ordered_selections(idxseqs, poss):
    if len(idxseqs) == 0:
        return 1
    if len(idxseqs) == 1:
        return len(idxseqs[0]) - poss[0]
    poss = list(poss) 
    tot = 0
    for na in idxseqs[0][poss[0]:]:
        for i in range(1, len(poss)):
            while poss[i] < len(idxseqs[i]) and idxseqs[i][poss[i]] <= na:
                poss[i] += 1
            if poss[i] >= len(idxseqs[i]):
                return tot
        tot += count_ordered_selections(idxseqs[1:], poss[1:])
    return tot


def check_fair(w):
    wa=np.array(list(w))
    dice = sorted(set(wa))
    indices = {d: np.arange(len(wa))[wa==d] for d in dice}
    counts = {}
    for perm in itertools.permutations(dice):
        idxseqs = [indices[d] for d in perm]
        counts[perm] = count_ordered_selections(idxseqs, [0 for i in idxseqs])
    return len(set(counts.values())) == 1

_REs = {}

def restrict_word(word, subset):
    if not subset:
        return ""
    if subset not in _REs:
        pat = re.compile(f"[^{subset}]")
        _REs[subset] = pat
    else:
        pat = _REs[subset]
    return pat.sub("", word)

class FairDices:
    def __init__(self, dice_sizes: dict):
        # dice_sizes: {'A': 4, 'B': 6, ...}
        self.dice_sizes = dice_sizes
        self.dice_names = ''.join(sorted(dice_sizes.keys()))
        self.n = len(self.dice_sizes)
        self.range = sum(self.dice_sizes.values())
        self.words = set()
        self.prefixes = set([""])

        if self.n == 0:
            self.insert("")
        if self.n == 1:
            self.insert(self.dice_names * self.dice_sizes[self.dice_names])
        # otherwise needs compute_from

    def compute_from(self, from0: 'FairDices', from1: 'FairDices', check_subdice: tuple=()):
        "Assumes all smaller FairDices are built"
        if self.n < 2:
            return
        total_throws = np.prod(list(self.dice_sizes.values()))
        if total_throws % math.factorial(self.n) != 0:
            print(f"No solution: total number of throw outcomes not divisible by n!={math.factorial(self.n)}")
        outcomes_per_permutation = total_throws // math.factorial(self.n)

        bin_index = restrict_word(from0.dice_names, from1.dice_names)
        bins0 = defaultdict(list)
        for w in from0.words:
            bins0[restrict_word(w, bin_index)].append(w)
        bins1 = defaultdict(list)
        for w in from1.words:
            bins1[restrict_word(w, bin_index)].append(w)

        allbins = set(bins0.keys()).union(bins1.keys())
        bins0sizes = np.array([len(bins0.get(b, ())) for b in allbins])
        bins1sizes = np.array([len(bins1.get(b, ())) for b in allbins])

        print(f"Combining {from0.dice_names!r} ({len(from0.words)} words) and {from1.dice_names!r} ({len(from1.words)} words) with {len(bins0)} bins on dices {bin_index!r}")
        qs = [0., 0.01, 0.25, 0.50, 0.75, 0.99, 1.]
        print(f"""Bin size quantiles (min, 1%, 25%, 50%. 75%, 99%, max):
    {from0.dice_names!r}: {np.quantile(bins0sizes, qs)}
    {from1.dice_names!r}: {np.quantile(bins1sizes, qs)}
    product: {np.quantile(bins0sizes * bins1sizes, qs)}""")
        
        def matches_subdice(w):
            for sd in check_subdice:
                if restrict_word(w, sd.dice_names) not in sd.prefixes:
                    return False
            return True

        interleaved = set()
        def interleave(done, rem0, rem1):
            #print(" "*len(done), done, rem0, rem1)
            if not matches_subdice(done):
                return

            if not rem0 and not rem1:
                interleaved.add(done)
                return

            if not rem0:
                interleave(done + rem1, "", "")
                return

            if not rem1:
                interleave(done + rem0, "", "")
                return

            if rem0[0] == rem1[0]:
                interleave(done + rem0[0], rem0[1:], rem1[1:])
                return
    
            # rem0[0] != rem1[0]
            if rem0[0] in bin_index and rem1[0] in bin_index:
                assert False
            if rem0[0] in bin_index:
                interleave(done + rem1[0], rem0, rem1[1:])
                return
            if rem1[0] in bin_index:
                interleave(done + rem0[0], rem0[1:], rem1)
                return

            interleave(done + rem0[0], rem0[1:], rem1)
            interleave(done + rem1[0], rem0, rem1[1:])

        tr = tqdm.tqdm(total=np.sum(bins0sizes*bins1sizes), desc="interleave", position=0, leave=True)
        for bi in bins0:
            for w0 in bins0[bi]:
                for w1 in bins1[bi]:
                    #l0 = len(interleaved)
                    interleave("", w0, w1)
                    #tr.write(f"{w0} {w1} {len(interleaved) - l0}")
                tr.update(len(bins1[bi]))
        tr.close()
        
        print(f"Found {len(interleaved)} interleaved candidates consistent with subdices {[sd.dice_names for sd in check_subdice]}")
        for w in tqdm.tqdm(interleaved, desc="check fairness", position=0, leave=True):
            if check_fair(w):
                self.insert(w)
        print(f"Done: {self}")

    def __str__(self):
        return f"FairDices {self.dice_names!r} [{','.join(f'd{self.dice_sizes[d]}' for d in self.dice_names)}], {len(self.words)} words ({len(self.prefixes)} prefixes)"

    def insert(self, s):
        if s in self.words:
            return
        self.words.add(s)
        for i in range(len(s)):
            self.prefixes.add(s[:i+1])

    def relabel_copy(self, letters):
        letters = list(letters)
        assert sorted(letters) == letters
        assert len(letters) == self.n
        fd = FairDices({l: self.dice_sizes[d] for l, d in zip(letters, self.dice_names)})
        for w in self.words:
            wa0 = np.array(list(w))
            wa = wa0.copy()
            for l, d in zip(letters, self.dice_names):
                wa[wa0 == d] = l
            fd.insert(''.join(wa))
        return fd

In [2]:
def new_dice(ds, l, sides):
    ds[l] = FairDices({l: sides})
def combine(ds, d1, d2, checks=()):
    d12 = ''.join(sorted(set(d1).union(d2)))
    print(f"\n## Combining {d1} + {d2} -> {d12} (checking {checks})")
    ds[d12] = FairDices({l: ds[l].dice_sizes[l] for l in d12})
    ds[d12].compute_from(ds[d1], ds[d2], check_subdice=[ds[c] for c in checks])
def relabel(ds, src, tgt):
    for l0, l1 in zip(src, tgt):
        assert ds[l0].dice_sizes[l0] == ds[l1].dice_sizes[l1]
    ds[tgt] = ds[src].relabel_copy(tgt)

Ds = {}
new_dice(Ds, 'A', 6)
new_dice(Ds, 'B', 6)
new_dice(Ds, 'C', 12)
new_dice(Ds, 'D', 12)

combine(Ds, 'A', 'B')
combine(Ds, 'A', 'D')
combine(Ds, 'C', 'D')
relabel(Ds, 'AD', 'AC')
relabel(Ds, 'AD', 'BC')
relabel(Ds, 'AD', 'BD')

combine(Ds, 'AB', 'AC', ['BC'])
combine(Ds, 'AC', 'AD', ['CD'])
relabel(Ds, 'ABC', 'ABD')

combine(Ds, 'ABC', 'ABD', ['ACD'])


## Combining A + B -> AB (checking ())
Combining 'A' (1 words) and 'B' (1 words) with 1 bins on dices ''
Bin size quantiles (min, 1%, 25%, 50%. 75%, 99%, max):
    'A': [1. 1. 1. 1. 1. 1. 1.]
    'B': [1. 1. 1. 1. 1. 1. 1.]
    product: [1. 1. 1. 1. 1. 1. 1.]


interleave: 100%|██████████| 1/1 [00:00<00:00, 845.28it/s]


Found 924 interleaved candidates consistent with subdices []


check fairness: 100%|██████████| 924/924 [00:00<00:00, 22000.35it/s]


Done: FairDices 'AB' [d6,d6], 58 words (399 prefixes)

## Combining A + D -> AD (checking ())
Combining 'A' (1 words) and 'D' (1 words) with 1 bins on dices ''
Bin size quantiles (min, 1%, 25%, 50%. 75%, 99%, max):
    'A': [1. 1. 1. 1. 1. 1. 1.]
    'D': [1. 1. 1. 1. 1. 1. 1.]
    product: [1. 1. 1. 1. 1. 1. 1.]


...

Done: FairDices 'ABC' [d6,d6,d12], 1480 words (16232 prefixes)

## Combining AC + AD -> ACD (checking ['CD'])
Combining 'AC' (676 words) and 'AD' (676 words) with 1 bins on dices 'A'
Bin size quantiles (min, 1%, 25%, 50%. 75%, 99%, max):
    'AC': [676. 676. 676. 676. 676. 676. 676.]
    'AD': [676. 676. 676. 676. 676. 676. 676.]
    product: [456976. 456976. 456976. 456976. 456976. 456976. 456976.]


interleave: 100%|██████████| 456976/456976 [1:56:42<00:00, 65.26it/s]


Found 205318450 interleaved candidates consistent with subdices ['CD']


check fairness:  25%|██▍       | 50329855/205318450 [7:35:40<23:23:14, 1840.83it/s]


KeyboardInterrupt: 